In [2]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline

In [3]:
def logit(z): return 1./(1.+np.exp(-z))

In [4]:
def generate_players(num_of_players, skill_cap):
    ## returns a list of player's skill
    return [round(random.uniform(0,skill_cap),2) for i in range(num_of_players)]


def generate_games(players, num_of_games, scale=0.3, style='pystan'):

    if style == 'pystan':
        
        player1=[]
        player2=[]
        outcome=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )

            ##pystan player id is ONE based
            player1.append(p1+1)
            player2.append(p2+1)
            
            outcome.append(*random.choices([1,0],weights=[win_rate,1-win_rate]))

        return np.array([player1,player2,outcome])
    
    elif style == 'pygm':
        games=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )
            
            games.append((p1,p2,*random.choices([1,-1],weights=[win_rate,1-win_rate])))
        return games
    
    assert False 


In [5]:
def prediction_accuracy(skill_data, samples, true_players, num_valid_game=1000, scale=0.3):
    
    valid_games = generate_games(true_players, num_valid_game, scale, style='pystan')

    acc=0
    for g in range(num_valid_game):
        i,j,result=valid_games[0][g],valid_games[1][g],valid_games[2][g]

        prediction = logit( skill_data['scale']*(samples['skill'][:,i-1]-samples['skill'][:,j-1]) ).mean()

        acc+= int( (prediction >= 0.5 and result==1) or (prediction<0.5 and result==0) )
    return acc/num_valid_game

def prediction_accuracy_2(skill_data, samples, valid_data, scale=0.3):
    
    #valid_games = generate_games(true_players, num_valid_game, scale, style='pystan')

    acc=0
    for g in range(valid_data.shape[1]):
        p1,p2,result=valid_data[0][g],valid_data[1][g],valid_data[2][g]

        win_rate = logit( skill_data['scale']*(samples['skill'][:,p1-1]-samples['skill'][:,p2-1]) ).mean()
        
        predict_result = random.choices([1,0],weights=[win_rate,1-win_rate])

        if predict_result == result:
            acc += 1
    return acc/valid_data.shape[1]

In [6]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,10); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

Now, compile the model.  We'll save a version of it so we don't have to recompile every time.

In [7]:
import pickle
try:     # load it if already compiled
    sm = pickle.load(open('skill_model.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

We also need the observed data: number of players and games, which pairs played each game, and who won:

In [ ]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])

a[:,2:]


In [ ]:
scale=0.5
true_players_10 = generate_players(10,10)
games = generate_games(true_players_10, 1000, scale, style='pystan')
train_games = games[:,:700]
valid_games = games[:,700:]

In [ ]:
true_players_10

In [ ]:
skill_data = {
    'N': 10,
    'E': 700,
    'scale': scale,
    'win':train_games[2],
    'PA': train_games[0],
    'PB': train_games[1]
}


Now, we can perform MCMC on the model, and extract the samples:

In [ ]:
starting=time.time()
fit = sm.sampling(data=skill_data, iter=10000, chains=10)
print("spent:",round(time.time()-starting,3))

In [ ]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [ ]:
samples['skill'].mean(0)

In [ ]:
prediction_accuracy_2(skill_data,valid_games)

In [ ]:
print(result)

In [ ]:
def avg(x):
    return sum(x)/len(x)
for i in result:
    print(i,avg(result[i]))

In [ ]:
import  scipy
def plot_result(d):
    
    M = np.array([d[k] for k in d])
    x = np.array([k for k in d])
    y = np.mean(M, axis = 1)
    ystd = scipy.stats.sem(M, axis = 1)
    print(ystd)
    
    plt.figure()
    plt.scatter(x, y, color = 'black')
    plt.errorbar(x, y, yerr = ystd, color = 'black', capsize = 5)
    plt.show()
    
plot_result(result)
        

In [8]:

scale=0.5
num_of_players=50
result={}
for num_of_games in [10,50,100,500,700,1000,3000,5000]:
    starting_num_of_games=time.time()
    trials_result=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = generate_players(num_of_players,10)
        train_games = generate_games(true_players, num_of_games, scale, style='pystan')
#         print("true skills:",true_players)
        skill_data = {
            'N': num_of_players,
            'E': num_of_games,
            'scale': scale,
            'win':train_games[2],
            'PA': train_games[0],
            'PB': train_games[1]
        }
        starting=time.time()
        fit = sm.sampling(data=skill_data, iter=10000, chains=num_of_players)
        print("spent:",round(time.time()-starting,3))
        samples = fit.extract()
#         print("predicted skills:",samples['skill'].mean(0))
        acc=prediction_accuracy(skill_data, samples, true_players, num_valid_game=10000, scale=scale)
        print("acc:",acc,"\n")
        trials_result.append(acc)
        
    result[num_of_games]=trials_result
    print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
print(result)

starting num_of_games = 10


KeyboardInterrupt: 

In [ ]:
print(result)

In [ ]:
def avg(x):
    return sum(x)/len(x)
for i in result:
    print(i,avg(result[i]))

In [9]:
scale=0.5
num_of_players=10
result={}
#Generate games 
print("start generating")
Data = []
for trials in range(10):
    p = generate_players(num_of_players,50)
    games = generate_games(p,3000, scale, style='pystan')#####
    games_pool = games[:,:2500]
    valid_games = games[:,2500:]
    Data.append((p,games_pool,valid_games))


for num_of_games in [3,5,10,20,35,50,70,100,500,700,1000,2500]:
    starting_num_of_games=time.time()
    trials_result=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = Data[trials][0]
        
        train_games = Data[trials][1][:,:num_of_games]
        print("true skills:",true_players)
        skill_data = {
            'N': num_of_players,
            'E': num_of_games,
            'scale': scale,###
            'win':train_games[2],
            'PA': train_games[0],
            'PB': train_games[1]
        }
        starting=time.time()
        fit = sm.sampling(data=skill_data, iter=10000, chains=4)####
        print("spent:",round(time.time()-starting,3))
        samples = fit.extract()
        print("predicted skills:",samples['skill'].mean(0))
        acc=prediction_accuracy_2(skill_data,samples,Data[trials][2])
        print("acc:",acc,"\n")
        trials_result.append(acc)
        
    result[num_of_games]=trials_result
    print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
print(result)

scale=0.5
num_of_players=20
result2={}
#Generate games 
print("start generating")
Data = []
for trials in range(10):
    p = generate_players(num_of_players,50)
    games = generate_games(p,3000, scale, style='pystan')#####
    games_pool = games[:,:2500]
    valid_games = games[:,2500:]
    Data.append((p,games_pool,valid_games))


for num_of_games in [3,5,10,20,35,50,70,100,500,700,1000,2500]:
    starting_num_of_games=time.time()
    trials_result=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = Data[trials][0]
        
        train_games = Data[trials][1][:,:num_of_games]
        print("true skills:",true_players)
        skill_data = {
            'N': num_of_players,
            'E': num_of_games,
            'scale': scale,###
            'win':train_games[2],
            'PA': train_games[0],
            'PB': train_games[1]
        }
        starting=time.time()
        fit = sm.sampling(data=skill_data, iter=10000, chains=4)####
        print("spent:",round(time.time()-starting,3))
        samples = fit.extract()
        print("predicted skills:",samples['skill'].mean(0))
        acc=prediction_accuracy_2(skill_data,samples,Data[trials][2])
        print("acc:",acc,"\n")
        trials_result.append(acc)
        
    result2[num_of_games]=trials_result
    print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
print(result2)

scale=0.5
num_of_players=30
result3={}
#Generate games 
print("start generating")
Data = []
for trials in range(10):
    p = generate_players(num_of_players,50)
    games = generate_games(p,3000, scale, style='pystan')#####
    games_pool = games[:,:2500]
    valid_games = games[:,2500:]
    Data.append((p,games_pool,valid_games))


for num_of_games in [3,5,10,20,35,50,70,100,500,700,1000,2500]:
    starting_num_of_games=time.time()
    trials_result=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = Data[trials][0]
        
        train_games = Data[trials][1][:,:num_of_games]
        print("true skills:",true_players)
        skill_data = {
            'N': num_of_players,
            'E': num_of_games,
            'scale': scale,###
            'win':train_games[2],
            'PA': train_games[0],
            'PB': train_games[1]
        }
        starting=time.time()
        fit = sm.sampling(data=skill_data, iter=10000, chains=4)####
        print("spent:",round(time.time()-starting,3))
        samples = fit.extract()
        print("predicted skills:",samples['skill'].mean(0))
        acc=prediction_accuracy_2(skill_data,samples,Data[trials][2])
        print("acc:",acc,"\n")
        trials_result.append(acc)
        
    result3[num_of_games]=trials_result
    print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
print(result3)

scale=0.5
num_of_players=50
result4={}
#Generate games 
print("start generating")
Data = []
for trials in range(10):
    p = generate_players(num_of_players,50)
    games = generate_games(p,3000, scale, style='pystan')#####
    games_pool = games[:,:2500]
    valid_games = games[:,2500:]
    Data.append((p,games_pool,valid_games))


for num_of_games in [3,5,10,20,35,50,70,100,500,700,1000,2500]:
    starting_num_of_games=time.time()
    trials_result=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = Data[trials][0]
        
        train_games = Data[trials][1][:,:num_of_games]
        print("true skills:",true_players)
        skill_data = {
            'N': num_of_players,
            'E': num_of_games,
            'scale': scale,###
            'win':train_games[2],
            'PA': train_games[0],
            'PB': train_games[1]
        }
        starting=time.time()
        fit = sm.sampling(data=skill_data, iter=10000, chains=4)####
        print("spent:",round(time.time()-starting,3))
        samples = fit.extract()
        print("predicted skills:",samples['skill'].mean(0))
        acc=prediction_accuracy_2(skill_data,samples,Data[trials][2])
        print("acc:",acc,"\n")
        trials_result.append(acc)
        
    result4[num_of_games]=trials_result
    print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
print(result4)

start generating
starting num_of_games = 3
true skills: [43.07, 35.41, 37.75, 36.63, 2.45, 42.83, 45.92, 3.8, 32.58, 27.35]
spent: 1.659
predicted skills: [-0.01618265  0.02248983 -0.01267309 -0.01476362 -1.3157593  -1.31415195
  1.31368014 -1.31528588  1.30413561  1.31856586]
acc: 0.522 

true skills: [22.4, 29.61, 38.44, 26.1, 18.67, 20.84, 34.44, 39.77, 17.47, 18.58]
spent: 1.689
predicted skills: [ 0.00532221  0.00293121  2.19774503 -1.08374667 -1.08132027  1.30115047
 -0.00533094 -0.01242063  0.01602152 -1.32170922]
acc: 0.552 

true skills: [47.0, 20.45, 43.09, 47.38, 36.65, 20.44, 36.77, 30.59, 48.91, 15.52]
spent: 1.671
predicted skills: [-2.97763817e-02  2.02130298e-03  1.10432858e+00 -1.11519436e-02
  4.40529014e-03 -1.64182700e-02  1.09449359e+00 -2.19749287e+00
  1.35444755e+00 -1.31064336e+00]
acc: 0.548 

true skills: [45.64, 20.97, 40.61, 13.38, 16.01, 17.94, 5.38, 31.65, 40.78, 25.1]
spent: 1.655
predicted skills: [ 2.16790118  1.31835392 -0.00662805  0.0118435  -0.0070

spent: 1.88
predicted skills: [-1.68805244  0.34608924  4.28155139  0.81430221 -1.81834211 -1.03729091
  1.12756036  3.1866737  -1.83846617 -3.45668266]
acc: 0.682 

true skills: [47.0, 20.45, 43.09, 47.38, 36.65, 20.44, 36.77, 30.59, 48.91, 15.52]
spent: 1.845
predicted skills: [ 3.97753649 -3.0689191   1.62060487  1.79343225 -1.56786475  0.80322659
  0.42638795 -2.03645288  1.64838403 -3.59268266]
acc: 0.678 

true skills: [45.64, 20.97, 40.61, 13.38, 16.01, 17.94, 5.38, 31.65, 40.78, 25.1]
spent: 1.866
predicted skills: [ 4.2391144  -0.70553459  2.96380337  0.3442792  -2.47249794 -3.76553995
 -2.31945951  0.17188842  2.2084086  -0.6994277 ]
acc: 0.682 

true skills: [5.3, 2.01, 48.91, 34.79, 12.16, 24.2, 38.46, 32.71, 7.95, 32.25]
spent: 1.884
predicted skills: [-1.65388947 -2.4880551   4.79782362  1.70868041 -2.49140905 -0.92322334
  3.30057568  1.33793659 -2.28113877 -1.15190953]
acc: 0.684 

true skills: [18.67, 2.72, 13.49, 44.78, 28.87, 47.6, 22.32, 1.8, 22.46, 2.62]
spent: 1.8

spent: 2.572
predicted skills: [-2.61389386 -6.17215908  6.33215878  2.82383069 -3.58693627 -1.40926371
  5.1160497   1.21007624 -2.44057562  0.66687831]
acc: 0.804 

true skills: [18.67, 2.72, 13.49, 44.78, 28.87, 47.6, 22.32, 1.8, 22.46, 2.62]
spent: 2.356
predicted skills: [-0.70389247 -5.63544125 -2.84232621  5.90191381  3.5691564   5.12869865
  0.27888688 -3.84531334  1.08836772 -2.73345792]
acc: 0.812 

true skills: [24.01, 5.86, 47.62, 22.9, 10.09, 26.64, 30.95, 4.77, 42.81, 39.07]
spent: 2.356
predicted skills: [-1.02430172 -5.25347456  6.45516882 -1.34498002 -3.74823477  1.15660701
  1.12958919 -4.87506146  2.91872004  4.65424683]
acc: 0.808 

true skills: [16.03, 37.86, 15.06, 19.91, 31.62, 18.17, 15.01, 41.66, 6.57, 47.33]
spent: 2.352
predicted skills: [-1.41556107  1.44986391 -4.1949124   2.13734491  1.79434954 -1.48769007
 -4.04760116  5.20775425 -5.40255115  5.87407711]
acc: 0.816 

true skills: [3.31, 39.85, 7.0, 42.78, 0.53, 38.1, 33.88, 27.37, 18.56, 14.46]
spent: 2.4

spent: 12.408
predicted skills: [-1.88641842 -9.87081135 11.9460387  -2.04798684 -6.51168245  0.36540345
  3.15613695 -9.66492339  7.87201749  6.67747912]
acc: 0.906 

true skills: [16.03, 37.86, 15.06, 19.91, 31.62, 18.17, 15.01, 41.66, 6.57, 47.33]
spent: 12.428
predicted skills: [ -4.01826563   5.64588913  -5.21176309  -0.27593388   3.06430559
  -2.21369822  -5.04562013   7.71545608 -11.20715163  11.46441517]
acc: 0.872 

true skills: [3.31, 39.85, 7.0, 42.78, 0.53, 38.1, 33.88, 27.37, 18.56, 14.46]
spent: 12.634
predicted skills: [ -9.5361244    9.04436896  -6.50231677  10.77794098 -11.05126711
   6.9688179    3.73265752   1.2251437   -1.67565527  -3.1753104 ]
acc: 0.894 

true skills: [46.96, 14.53, 14.02, 17.07, 45.88, 42.96, 23.15, 38.82, 4.91, 30.85]
spent: 12.728
predicted skills: [ 10.6254847   -6.56746385  -6.53107682  -4.64746111   8.63316293
   6.51245292  -1.18406088   4.08552685 -12.17176274   1.14266623]
acc: 0.906 

Complete 700 | Spent: 126.02
starting num_of_games = 

spent: 1.972
predicted skills: [ 1.30941357e+00 -1.28372034e+00  6.18720590e-04  4.05802497e-03
 -1.31517382e+00 -1.63261450e-02 -4.47521301e-03 -9.73955088e-03
 -1.02641771e-02  1.04638615e-02 -1.79576234e-02 -1.31621029e+00
  1.53835720e-02  1.33864856e+00 -5.69778933e-03 -1.13347432e-02
  2.60111936e-02  1.32949505e+00 -1.54366835e-02 -1.51125566e-02]
acc: 0.554 

true skills: [35.37, 0.79, 38.77, 1.72, 12.77, 9.55, 7.48, 47.92, 28.54, 0.82, 25.13, 22.79, 19.84, 39.72, 17.46, 37.3, 13.79, 35.65, 2.99, 40.05]
spent: 1.924
predicted skills: [ 3.32813102e-03 -2.19731614e+00  1.57253107e-02 -4.59011721e-03
 -1.76310606e-02 -1.31266313e+00  6.65637475e-03 -5.24002492e-03
 -1.63151999e-02  1.07777668e+00 -1.70670667e-03  1.10444085e+00
 -2.38383322e-02  1.30129328e+00  1.40724972e-02 -3.10718875e-03
 -1.33271541e-03  9.66030228e-04  4.06074083e-03  4.52225500e-04]
acc: 0.512 

true skills: [24.34, 46.17, 49.89, 1.22, 29.6, 4.06, 21.9, 27.37, 26.3, 19.7, 46.81, 0.71, 16.39, 29.48, 16.84, 4

...

spent: 8.195
predicted skills: [ 1.4463513   0.48051871 -5.86980985  4.177781   -2.80262649 -3.35233171
 -6.39163341  1.83983444  0.25732277  5.2797781   0.66773344 -1.74488737
  7.25543628  0.78692433  6.09808863 -3.09845611  2.0214141   2.9913409
  0.28950667  0.05563729  3.29236981  4.57233459  1.32532179 -4.41288756
 -7.19349443  2.19899466 -2.88555674  4.78433532 -6.73779607 -6.74253588
  5.79151063  5.94456346 -1.29946462 -3.81965575  7.27251116 -4.65088284
 -0.08577699 -3.06157198  0.59943119  4.73918534 -3.82539824  5.82813319
 -3.30136491 -2.99775515 -6.46650671 -3.74352804 -1.03696062  0.50686574
  0.34208925  4.70013483]
acc: 0.794 

true skills: [32.54, 17.47, 38.41, 44.96, 47.38, 31.88, 3.64, 6.1, 34.22, 14.24, 47.09, 28.02, 32.0, 38.49, 6.74, 38.06, 10.82, 25.27, 36.09, 6.22, 1.72, 33.25, 16.04, 5.55, 29.6, 21.71, 39.75, 37.93, 27.01, 3.27, 13.85, 18.56, 28.83, 13.81, 29.38, 49.51, 43.05, 12.03, 1.4, 3.71, 8.2, 13.33, 18.77, 6.45, 9.38, 1.35, 47.41, 39.99, 40.97, 7.63]
sp

spent: 8.003
predicted skills: [ 1.00322583  7.29624574 -0.29086945  0.74567852 -7.80339561 -1.62582603
  0.37009667  5.41273878  4.13208029  2.9639133   2.23493182 -3.22875749
 -0.16727867 -5.30383665  1.23466032 -4.38287529 -1.7304266   4.35731458
  0.69310873 -5.47454049 -0.41419752 -4.20594419  4.46947052 -2.34554014
  3.41795229  5.82101609 -6.31441265 -5.36674201 -3.0670021   2.54045459
  4.11708656 -3.99129138 -2.44389045 -3.34707082  1.53043811 -2.5525976
  6.22162171  3.74988497  0.50408269 -0.75199674 -4.37527534  3.32288382
  4.94840096  3.12846494  5.1582813  -6.50811445  1.12293029 -7.25107758
 -3.14659836  5.8156064 ]
acc: 0.794 

Complete 500 | Spent: 87.97
starting num_of_games = 700
true skills: [30.69, 24.7, 1.36, 39.32, 18.09, 14.33, 4.22, 33.4, 25.14, 35.36, 28.16, 13.07, 48.92, 25.18, 46.12, 12.91, 36.7, 29.06, 27.11, 24.48, 35.38, 40.85, 27.06, 8.56, 0.58, 26.92, 16.18, 32.05, 1.11, 0.9, 43.81, 45.45, 17.18, 9.81, 49.79, 9.88, 25.02, 6.69, 25.36, 43.2, 14.32, 47.4

spent: 16.641
predicted skills: [-4.1457965  -2.2173154  -0.52138774 -5.78105419 -6.05143294  2.07681369
 -3.79271289  1.01198217  5.77503371 -1.29101635 -0.1352404   1.62796937
 -2.24799761 -3.82533919  2.98169801 -4.71130205 -0.07570481 -6.73972709
  3.47912093  7.47019447 -9.57505872  3.0368783   6.4231792  -6.35845664
 -5.2002877   5.66000476  0.31880324 -0.08734158 -3.65225118  4.88595531
 -3.91525156  1.95288543  5.5925089   6.8456794  -4.43808833 -3.74327478
 -7.63871956  8.23118556 -3.50085416  2.71949866 -2.35425773  6.60881501
  2.60969512 -5.34504024  1.44393315  7.08012536  0.90685074  2.52175781
  3.86310685  2.24504541]
acc: 0.836 

true skills: [35.25, 47.67, 23.51, 26.34, 0.48, 10.68, 26.03, 41.94, 37.94, 32.41, 32.96, 19.36, 22.8, 4.15, 25.83, 9.64, 21.54, 38.9, 32.93, 5.79, 28.32, 4.42, 40.38, 22.18, 34.61, 43.47, 3.41, 2.53, 18.55, 34.75, 39.02, 6.21, 14.77, 16.48, 36.42, 17.42, 46.34, 37.24, 31.58, 22.31, 4.22, 36.2, 41.04, 37.93, 43.3, 0.47, 33.03, 1.34, 14.41, 41.

spent: 15.757
predicted skills: [-1.69911929  3.51077939 -2.63847362 -5.49226478  7.5177581  -3.49403033
 -7.94227224  2.65504759  2.89134736  8.30701565 -5.09014721  6.90138879
 -5.10386054  3.14542496  7.5775844  -0.57619742 -8.0878264   2.01424362
  5.39869796 -3.56996184  5.90647883  1.668785   -3.46725655  9.64864501
  0.8048615  -8.3435101  -0.82394395  5.57697746 -0.42141476  4.19395518
  5.66191184  0.84703929  7.34287719 -0.45491956 -5.91604778 -5.6305084
  5.31416829  0.67711889  0.34595    -6.22264247 -0.54925876  4.20253239
  7.15407665 -5.14396992 -7.5062738  -3.53055737 -9.07792334 -4.02106008
 -3.77010707 -1.07715387]
acc: 0.84 

true skills: [14.28, 18.19, 23.58, 7.05, 5.56, 38.33, 15.33, 32.35, 44.45, 26.03, 31.31, 28.76, 25.27, 10.26, 40.27, 13.52, 23.41, 5.61, 42.16, 49.51, 2.43, 38.34, 48.57, 3.08, 5.74, 43.74, 27.21, 27.14, 19.56, 45.57, 18.61, 29.39, 42.09, 49.41, 13.64, 13.52, 1.17, 48.37, 16.0, 43.55, 18.13, 47.07, 35.51, 6.56, 29.69, 48.94, 32.57, 34.71, 36.56,

spent: 36.865
predicted skills: [ -7.20982004 -10.19610134   1.28751526  -0.87284028  -5.92446047
  -0.54340246  -1.42601034  -1.35011669   3.71899581  12.26374363
 -13.55422291  -3.72854996   5.88382723  -5.07956803   2.04582228
   5.48889107 -11.81172165   2.19743411  -3.02683873   7.63110471
   2.6463692    2.01195253   7.15284045  -5.6013974    8.14139075
  10.23131314  -2.84559869  10.26189584  -0.74569203   0.6826107
   5.73912831   9.24814611  -6.8323685   -7.35449877  -6.04381927
  -9.59735674  -2.3498036   -8.77739643  -4.13855619   3.74489835
 -11.25291933   2.28430871   7.14569665   8.07096523   9.84051784
  10.80707972   0.98904539  -2.66340173   3.76708362 -10.35034679]
acc: 0.89 

true skills: [17.79, 6.06, 29.02, 23.28, 21.86, 24.72, 24.19, 10.68, 24.65, 43.48, 33.57, 23.77, 31.99, 35.41, 34.9, 2.28, 12.04, 36.61, 42.13, 2.11, 38.23, 5.75, 26.88, 6.26, 7.8, 24.87, 20.36, 29.64, 40.28, 5.94, 42.37, 9.2, 13.9, 42.04, 1.54, 3.34, 44.73, 13.94, 20.22, 20.69, 34.1, 34.51, 28.